# Test 2
## Imports and configurations 
 (global variables, gecko...)

In [1]:
# Importing required libraries for web scrapping
import time

from bs4 import BeautifulSoup
from selenium import webdriver

import os

In [2]:
jar = "./pos-tagger/stanford-postagger/stanford-postagger-4.2.0.jar"
model = "./pos-tagger/stanford-postagger/models/spanish-ud.tagger"
os.environ["JAVAHOME"] = "/usr/bin/java"

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://alianzaverde.org.co/lideres-verdes/congreso-2022/senado-2022'
IMAGE_URL_ROOT = 'https://alianzaverde.org.co'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

## Webscraping

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\BORISL~1\AppData\Local\Temp/ipykernel_11224/1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    if relative_path!="":
        browser.get(relative_path)
    else:
        browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
candidates = []

In [8]:
# Getting HTML content for a particular news listing page

# Controlling the page which will be showing cards for every cycle

soup = make_request(browser, "")

# Finding the section where news are contained 
layout = soup.find(class_ = 'page-1246')

# Getting blocks from layout
blocks = layout.find_all(class_ = 'sppb-wow')#, recursive = True)

for block in blocks:

    # Finding and concatenating news cards
    cards = block.find_all(class_ = 'sppb-col-md-3')
    
    # Reviewing a maximun of two cards by page (in order to mantain low processing power for the exercise)
    for i in range(len(cards)):
        try:
            description = cards[i].find(class_ = 'sppb-person-designation').get_text()
            
            image = cards[i].find('img', class_ = 'sppb-img-responsive')['src']
        
            group_network = cards[i].find('ul', class_ = 'sppb-person-social')
            networks = {}
            if group_network is not None:
                items_network = group_network.find_all('li')
                for item in items_network:
                    networks.update({item.find('a')['aria-label']:item.find('a')['href']})

            candidates.append({
                'image': IMAGE_URL_ROOT+image,
                'description': description,
                'networks' : networks,
            })
        except:
            print("")
        

In [9]:
candidates

[{'image': 'https://alianzaverde.org.co/images/Candidatos_2022/senado/2.jpg',
  'description': 'Abogada y filósofa, con un máster en Ciencias de la Administración Pública, en Alemania. Experta en gestión pública, descentralización, transparencia, buen gobierno, modernización del estado, cultura ciudadana y cooperación internacional, temas en los cuales se ha desempeñado alternativamente como servidora pública, consultora y docente.',
  'networks': {'Facebook': 'https://twitter.com/VivianaBarbere ',
   'Twitter': 'https://twitter.com/VivianaBarbere',
   'Instagram': 'https://www.instagram.com/barberenaviviana/ '}},
 {'image': 'https://alianzaverde.org.co/images/Candidatos_2022/senado/3.jpg',
  'description': 'Nacida de seres excepcionales, en tiempos inciertos; es la mujer de retos resueltos. Desde el inicio de su carrera política, su liderazgo se enfoca en mantener la cultura de la equidad, la familia y el fortalecimiento económico, político y social de las regiones en Colombia',
  'ne